In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
import random

import math # cos() for Rastrigin
import copy # array-copying convenience
import sys # max float
import sympy as sp
from sympy.plotting import plot
from sympy.plotting import plot3d
import matplotlib.pyplot as plt
import time
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import deque
from ddpg_agent import Agent

In [2]:
## parameter/ constant
bandwidth = 1e6  # [hz]
Speed_of_light = 3e8  # [m/s]
Carrier_freq = 2e9  # [hz]=1/s
GBS_location = [0,0,20] #[m] 20
Num_GBS = 1 
NUM_User = 10
NUM_UAV = 2
Cell_Radius = 1000 #[m]
UAV_Height = 100 #[m] "Cell-Edge User Offloading via Flying UAV in Non-Uniform Heterogeneous Cellular Networks," in IEEE Transactions on Wireless Communications, vol. 19, no. 4, pp. 2411-2426, April 2020, doi: 10.1109/TWC.2020.2964656.
P_UAV = 10 **(37 / 10) / 1000 #[watt] 
P_GBS = 10 **(40 / 10) / 1000 #[watt]
P_MAX = 0.4 # 26dbm, user
# P_MIN = 0.1 # 20 dbm
epsilon = 0.38 # power amplifier efficiency
AWGN_DBM = -174 #[dBm]
AWGN_W = 10 ** (AWGN_DBM / 10) / 1000
air_density = 1.225 # Air density at sea level (kg/m³)
g = 9.81            # Gravitational acceleration (m/s²)
mass = 0.5           # Mass of UAV in kg
radius = 0.2         # Rotor radius in meters
num_rotors = 4
P_MOVE =10 #[watt]
V = 20 # m/s


In [3]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [4]:
# Multi-User Data Generation with Poisson Distribution
def generate_multi_user_data(num_users=5, base_lambda=10, num_hours=3*720, start_date=None, seed=42):
    """
    Generate synthetic hourly traffic data for multiple users using Poisson distribution.

    Args:
        num_users: Number of users to generate data for
        base_lambda: Base lambda for Poisson distribution (Mbps avg Rate)
        num_hours: Number of hours to generate
        start_date: Starting date for the data
        seed: Random seed for reproducibility

    Returns:
        df: DataFrame with timestamps and traffic values for all users
    """
    # Set random seed for reproducibility
    random.seed(seed)
    np.random.seed(seed)

    days_in_month = 2 * 30  # Length of the monthly pattern

    # Common daily pattern - all users follow similar daily rhythms (low at night, high in evening)
    base_daily_pattern = np.array([
        0.05, 0.05, 0.05, 0.05, 0.05, 0.05,  # Midnight to 6 AM (low traffic)
        0.10, 0.20, 0.35, 0.50, 0.65, 0.80,  # Morning to afternoon (moderate traffic)
        0.90, 0.98, 1.00, 0.98, 0.90, 0.80,  # Evening peak hours (high traffic)
        0.65, 0.50, 0.35, 0.20, 0.10, 0.05   # Late night (declining traffic)
    ])

    # Base weekly pattern - all users follow similar weekly patterns
    base_weekly_pattern = np.array([0.9, 0.7, 0.7, 0.6, 0.8, 1.0, 1.0])  # Mon-Sun

    # Base monthly pattern - common seasonal effect
    base_monthly_pattern = 1 - 0.2 * np.sin(2 * np.pi * np.arange(days_in_month) / days_in_month)

    # Set the start date to 2 months ago from today if not provided
    if start_date is None:
        start_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=60)

    # Generate timestamps (same for all users)
    timestamps = []
    current_time = start_date
    for _ in range(num_hours):
        timestamps.append(current_time)
        current_time += timedelta(hours=1)

    # Create user-specific patterns with variations
    user_data = {}
    user_patterns = {}

    for user_id in range(1, num_users + 1):
        # Create user-specific variations of the patterns
        user_lambda = base_lambda * random.uniform(0.5, 2.0)  # Different baseline traffic volume

        # Daily pattern variation (shift peak hours slightly, change intensities)
        shift = random.randint(-2, 2)  # Shift peak by -2 to +2 hours
        daily_pattern = np.roll(base_daily_pattern, shift)
        daily_pattern = daily_pattern * random.uniform(0.8, 1.2)  # Vary intensity
        daily_pattern = np.clip(daily_pattern, 0.01, 1.0)  # Ensure values are reasonable

        # Weekly pattern variation (some users more active on weekends, others on weekdays)
        weekly_variation = np.random.uniform(0.8, 1.2, size=7)
        weekly_pattern = base_weekly_pattern * weekly_variation
        weekly_pattern = np.clip(weekly_pattern, 0.3, 1.5)  # Ensure values are reasonable

        # Monthly pattern might have different phases for different users
        phase_shift = random.uniform(0, 2*np.pi)
        monthly_pattern = 1 - 0.2 * np.sin(2 * np.pi * np.arange(days_in_month) / days_in_month + phase_shift)

        # Store patterns for reference
        user_patterns[user_id] = {
            "lambda": user_lambda,
            "daily": daily_pattern,
            "weekly": weekly_pattern,
            "monthly": monthly_pattern
        }

        # Generate traffic values for this user
        values = []
        for t in timestamps:
            hour = t.hour
            day_of_week = t.weekday()
            day_of_month = (t - start_date).days % days_in_month

            # Combine patterns to adjust lambda
            adjusted_lambda = (
                user_lambda *
                daily_pattern[hour] *
                weekly_pattern[day_of_week] *
                monthly_pattern[day_of_month]
            )

            # Add small random trend component
            trend_factor = 1.0 + 0.0005 * (t - start_date).days  # Slight growth over time
            adjusted_lambda *= trend_factor

            # Generate traffic using Poisson distribution
            value = np.random.poisson(adjusted_lambda)
            values.append(value)

        user_data[user_id] = values

    # Create DataFrame with all users
    df_data = {"timestamp": timestamps}
    for user_id, values in user_data.items():
        df_data[f"user_{user_id}"] = values

    df = pd.DataFrame(df_data)

    return df, user_patterns

In [5]:
df, user_patterns = generate_multi_user_data(num_users=5, base_lambda=10, num_hours=3*720, start_date=None, seed=42)

In [6]:
df

,timestamp,user_1,user_2,user_3,user_4,user_5
0,2025-06-21 00:00:00,1,0,0,0,2
1,2025-06-21 01:00:00,1,2,0,1,1
2,2025-06-21 02:00:00,2,0,0,1,0
3,2025-06-21 03:00:00,0,1,0,1,0
4,2025-06-21 04:00:00,0,0,1,1,0
...,...,...,...,...,...,...
2155,2025-09-18 19:00:00,4,2,2,0,3
2156,2025-09-18 20:00:00,0,0,2,1,2
2157,2025-09-18 21:00:00,2,0,0,4,6
2158,2025-09-18 22:00:00,0,1,2,1,7


In [7]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_len=10):
        """Initialize positional encoding."""
        super(PositionalEncoding, self).__init__()

        # Create positional encoding matrix
        pe = torch.zeros(max_seq_len, d_model)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        # Apply sin/cos functions
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # Add batch dimension and register as buffer (not a parameter)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """Add positional encoding to input."""
        return x + self.pe[:, :x.size(1), :]

In [8]:
# Multi-Head Attention Module
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        """Initialize multi-head attention."""
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        # Linear projections for Q, K, V
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)

        # Output projection
        self.W_o = nn.Linear(d_model, d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, d_k)."""
        # x shape: [batch_size, seq_len, d_model]
        x = x.view(batch_size, -1, self.num_heads, self.d_k)
        # Transpose to [batch_size, num_heads, seq_len, d_k]
        return x.transpose(1, 2)

    def forward(self, query, key, value, mask=None):
        """Forward pass for multi-head attention."""
        batch_size = query.size(0)

        # Linear projections
        q = self.W_q(query)  # [batch_size, seq_len_q, d_model]
        k = self.W_k(key)    # [batch_size, seq_len_k, d_model]
        v = self.W_v(value)  # [batch_size, seq_len_v, d_model]

        # Split heads
        q = self.split_heads(q, batch_size)  # [batch_size, num_heads, seq_len_q, d_k]
        k = self.split_heads(k, batch_size)  # [batch_size, num_heads, seq_len_k, d_k]
        v = self.split_heads(v, batch_size)  # [batch_size, num_heads, seq_len_v, d_k]

        # Scaled dot-product attention
        # matmul: [batch_size, num_heads, seq_len_q, seq_len_k]
        scores = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.d_k, dtype=torch.float32))

        # Apply mask if provided
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        # Apply softmax
        attention_weights = torch.softmax(scores, dim=-1)

        # Apply attention weights to values
        # context: [batch_size, num_heads, seq_len_q, d_k]
        context = torch.matmul(attention_weights, v)

        # Transpose and concatenate heads
        # context: [batch_size, seq_len_q, num_heads, d_k]
        context = context.transpose(1, 2).contiguous()

        # context: [batch_size, seq_len_q, d_model]
        context = context.view(batch_size, -1, self.d_model)

        # Final linear projection
        output = self.W_o(context)  # [batch_size, seq_len_q, d_model]

        return output


In [9]:
# Transformer Encoder Block
class EncoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        """Initialize the encoder block."""
        super(EncoderBlock, self).__init__()

        # Multi-head attention
        self.self_attention = MultiHeadAttention(d_model, num_heads)

        # Feed-forward network
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )

        # Layer normalization and dropout
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        """Forward pass for encoder block."""
        # Self-attention with residual connection and layer norm
        attn_output = self.self_attention(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))

        # Feed-forward network with residual connection and layer norm
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))

        return x

In [10]:
# Complete Transformer Encoder
class TransformerEncoder(nn.Module):
    def __init__(self, input_dim, d_model, num_heads, d_ff, num_layers, dropout=0.1, max_seq_len=1000):
        """Initialize the transformer encoder."""
        super(TransformerEncoder, self).__init__()

        # Input projection
        self.input_projection = nn.Linear(input_dim, d_model)

        # Positional encoding
        self.positional_encoding = PositionalEncoding(d_model, max_seq_len)

        # Dropout
        self.dropout = nn.Dropout(dropout)

        # Stack of encoder layers
        self.encoder_layers = nn.ModuleList([
            EncoderBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, x, mask=None):
        """Forward pass for transformer encoder."""
        # Project input to d_model dimensions
        x = self.input_projection(x)  # [batch_size, seq_len, d_model]

        # Add positional encoding
        x = self.positional_encoding(x)

        # Apply dropout
        x = self.dropout(x)

        # Pass through encoder layers
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x, mask)

        return x  # [batch_size, seq_len, d_model]

In [11]:
# Centralized Multi-User Traffic Forecasting Model
class CentralizedTrafficForecastingModel(nn.Module):
    def __init__(self,
                 num_users,
                 input_dim=1,
                 d_model=128,
                 num_heads=8,
                 d_ff=256,
                 num_layers=4,
                 dropout=0.1,
                 pred_horizon=24):
        """
        Initialize centralized traffic forecasting model for multiple users.

        Args:
            num_users: Number of users to predict for
            input_dim: Input dimension per user (usually 1 for traffic data)
            d_model: Hidden dimension of the model
            num_heads: Number of attention heads
            d_ff: Feed-forward dimension
            num_layers: Number of encoder layers
            dropout: Dropout rate
            pred_horizon: Number of time steps to predict (24 for a day)
        """
        super(CentralizedTrafficForecastingModel, self).__init__()

        # Store number of users
        self.num_users = num_users

        # Store input dimensions
        self.input_dim = input_dim

        # Transformer encoder - takes multi-user input
        self.encoder = TransformerEncoder(
            input_dim=input_dim * num_users,  # Input has data for all users
            d_model=d_model,
            num_heads=num_heads,
            d_ff=d_ff,
            num_layers=num_layers,
            dropout=dropout
        )

        # Global average pooling
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1) # average value (a,b,c) ==> (a,b,1)

        # Output projection - produces predictions for all users
        self.output_layer = nn.Linear(d_model, pred_horizon * num_users)

    def forward(self, x):
        """
        Forward pass for centralized traffic forecasting model.

        Args:
            x: Input tensor of shape [batch_size, seq_len, input_dim * num_users]
               where each feature dimension contains data for one user

        Returns:
            Predictions for all users of shape [batch_size, pred_horizon * num_users]
        """
        # Run through encoder
        enc_output = self.encoder(x)  # [batch_size, seq_len, d_model]

        # Global average pooling
        pooling_input = enc_output.transpose(1, 2)  # [batch_size, d_model, seq_len]
        avg_representation = self.global_avg_pool(pooling_input).squeeze(-1)  # [batch_size, d_model]

        # Final output projection - produces predictions for all users
        output = self.output_layer(avg_representation)  # [batch_size, pred_horizon * num_users]

        return output

In [12]:
# Custom Dataset for multi-user time series forecasting
class MultiUserTrafficDataset(Dataset):
    def __init__(self, X, y):
        """Initialize dataset for multi-user data."""
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [13]:
input_arrays = np.array([
   [[0.01], [0.2], [0.01], [0.6], [0.01], [0.8], [0.4], [0.01], [1.0]],  # user_1's transformed data
   [[0.1], [0.4], [0.3], [0.0], [0.2], [0.35], [0.05], [0.25], [0.15]],   # user_2's transformed data
   [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]   ] )    # user_3's zero array
input_arrays = np.stack(input_arrays)


In [14]:
model_input = input_arrays.reshape(1, 27, -1)

In [15]:
# Data Processor for Multi-User Traffic Forecasting
class CentralizedDataProcessor:
    def __init__(self, df, user_cols, timestamp_col='timestamp'):
        """
        Initialize data processor for multiple users.

        Args:
            df: DataFrame with traffic data
            user_cols: List of column names for user traffic data
            timestamp_col: Name of the timestamp column
        """
        self.df = df.copy()
        self.user_cols = user_cols
        self.timestamp_col = timestamp_col
        self.num_users = len(user_cols)

        # Initialize a scaler for each user
        self.scalers = {}
        for user_col in user_cols:
            self.scalers[user_col] = MinMaxScaler(feature_range=(0, 1))
            # Fit scaler on this user's traffic data
            user_data = self.df[user_col].values.reshape(-1, 1)
            if len(user_data) > 0:  # Check if data is not empty
                self.scalers[user_col].fit(user_data)

        # Ensure timestamp is datetime
        if not pd.api.types.is_datetime64_any_dtype(self.df[timestamp_col]):
            self.df[timestamp_col] = pd.to_datetime(self.df[timestamp_col])

        # Sort by timestamp
        self.df = self.df.sort_values(by=timestamp_col)

    def prepare_input_for_date(self, target_date, seq_length=120):
        """
        Prepare input data for predicting traffic on a specific date for all users.

        Args:
            target_date: The date to predict (datetime or string)
            seq_length: Fixed sequence length for model input

        Returns:
            model_input: Normalized input tensor for the model
            target_hours: Hours to be predicted
        """
        if isinstance(target_date, str):
            target_date = pd.to_datetime(target_date)

        # Get data from previous 3 days (72 hours)
        three_days_ago = target_date - timedelta(days=3)
        prev_3_days = self.df[(self.df[self.timestamp_col] >= three_days_ago) &
                              (self.df[self.timestamp_col] < target_date)]

        # Get data from same day of the week for the past 2 weeks
        day_of_week = target_date.weekday()

        # First previous week
        week_ago = target_date - timedelta(days=7)
        week_ago_day = week_ago.replace(hour=0, minute=0, second=0, microsecond=0)
        prev_week_same_day = self.df[(self.df[self.timestamp_col] >= week_ago_day) &
                                     (self.df[self.timestamp_col] < week_ago_day + timedelta(days=1))]

        # Second previous week
        two_weeks_ago = target_date - timedelta(days=14)
        two_weeks_ago_day = two_weeks_ago.replace(hour=0, minute=0, second=0, microsecond=0)
        prev_2_weeks_same_day = self.df[(self.df[self.timestamp_col] >= two_weeks_ago_day) &
                                        (self.df[self.timestamp_col] < two_weeks_ago_day + timedelta(days=1))]

        # Combine all data
        input_data_frames = [prev_3_days, prev_week_same_day, prev_2_weeks_same_day]
        all_input_df = pd.concat(input_data_frames)
        all_input_df = all_input_df.sort_values(by=self.timestamp_col)

        if all_input_df.empty:
            raise ValueError(f"No input data available for {target_date}")

        # Prepare multi-user input
        # Scale data for each user and concatenate horizontally
        input_arrays = []
        for user_col in self.user_cols:
            # Skip if no data for this user
            if user_col in all_input_df.columns:
                # Replace zeros with a small non-zero value (e.g., 0.01)
                all_input_df[user_col] = all_input_df[user_col].replace(0, 0.01)
            if user_col not in all_input_df.columns:
                # Create array of zeros for this user
                user_array = np.zeros((len(all_input_df), 1)) # (row, column)
            else:
                # Scale this user's data
                try:
                    user_data = all_input_df[user_col].values.reshape(-1, 1)
                    if len(user_data) > 0:
                        user_array = self.scalers[user_col].transform(user_data)
                    else:
                        user_array = np.zeros((len(all_input_df), 1))
                except Exception as e:
                    print(f"Error scaling data for {user_col}: {e}")
                    user_array = np.zeros((len(all_input_df), 1))

            input_arrays.append(user_array)
        # Concatenate all user data horizontally
        if input_arrays:
            input_values = np.hstack(input_arrays)
        else:
            # If no user data available, create zero array
            input_values = np.zeros((len(all_input_df), self.num_users))

        # Handle sequence length consistency
        actual_length = input_values.shape[0] #sequence leangth
        if actual_length > seq_length:
            # If we have too much data, take the most recent seq_length values
            input_values = input_values[-seq_length:]
        elif actual_length < seq_length:
            # If we don't have enough data, pad with zeros at the beginning (focus on current data)
            padding = np.zeros((seq_length - actual_length, input_values.shape[1]))
            input_values = np.vstack((padding, input_values))

        # Get target hours for prediction (24 hours starting from target_date)
        target_hours = [target_date + timedelta(hours=i) for i in range(24)]

        # before input_values [seq_len, features=num_users]
        # Reshape to model input format: [batch_size=1, seq_len, features=num_users]
        model_input = input_values.reshape(1, seq_length, -1) #add one more dimension of batch_size=1, to match the input format
        return model_input, target_hours

    def prepare_training_data(self, start_date, end_date, seq_length=120, pred_horizon=24):
        """
        Prepare training data for all users in a date range.

        Args:
            start_date: Start of training period
            end_date: End of training period
            seq_length: Input sequence length
            pred_horizon: Prediction horizon (usually 24 hours)

        Returns:
            X_train: Training inputs for all users combined
            y_train: Training targets for all users combined
        """
        if isinstance(start_date, str):
            start_date = pd.to_datetime(start_date)
        if isinstance(end_date, str):
            end_date = pd.to_datetime(end_date)

        X = []
        y = []
        
        # Process each day in the date range
        current_date = start_date
        while current_date <= end_date:
            try:
                # Get input data for this date (all users)
                model_input, _ = self.prepare_input_for_date(current_date, seq_length)

                # Get target data (next 24 hours from current_date) for all users
                target_start = current_date
                target_end = target_start + timedelta(days=1)
                target_data = self.df[(self.df[self.timestamp_col] >= target_start) &
                                     (self.df[self.timestamp_col] < target_end)]

                # Only use days with complete target data (24 hours)
                if len(target_data) == pred_horizon:
                    # Scale target data for each user and concatenate
                    target_arrays = []
                    for user_col in self.user_cols:
                        if user_col in target_data.columns:
                            user_data = target_data[user_col].values.reshape(-1, 1)
                            if len(user_data) > 0:
                                user_array = self.scalers[user_col].transform(user_data)
                                target_arrays.append(user_array.flatten())
                            else:
                                target_arrays.append(np.zeros(pred_horizon))
                        else:
                            target_arrays.append(np.zeros(pred_horizon))

                    # Concatenate all target arrays
                    target_values = np.concatenate(target_arrays)
                    # print("target_values",target_values)

                    # Add to training sets
                    X.append(model_input[0])  # Shape: [seq_length, num_users]
                    y.append(target_values)  # Shape: [pred_horizon * num_users]
                else:
                    print(f"Skipping {current_date.date()} - incomplete target data ({len(target_data)} hours)")

            except Exception as e:
                print(f"Error preparing data for {current_date.date()}: {e}")

            # Move to next day
            current_date += timedelta(days=1)

        # Convert lists to numpy arrays
        if not X:
            raise ValueError("No valid training samples were generated")

        # Convert to numpy arrays using stack to ensure consistent shapes
        X = np.stack(X)
        y = np.stack(y)

        print(f"Prepared {len(X)} training samples")
        print(f"X shape: {X.shape}, y shape: {y.shape}")

        return X, y

    def inverse_transform_predictions(self, predictions):
        """
        Convert scaled predictions back to original scale for all users.

        Args:
            predictions: Scaled predictions from the model of shape [pred_horizon * num_users]

        Returns:
            Dictionary of original scale predictions for each user
        """
        pred_horizon = 148  # Assuming 24-hour prediction
        user_predictions = {}

        for i, user_col in enumerate(self.user_cols):
            # Extract this user's predictions
            user_start = i * pred_horizon
            user_end = (i + 1) * pred_horizon
            user_pred_scaled = predictions[user_start:user_end]
            # Reshape for inverse transform
            user_pred_scaled = user_pred_scaled.reshape(-1, 1)

            # Inverse transform
            user_pred = self.scalers[user_col].inverse_transform(user_pred_scaled).flatten()

            # Store in dictionary
            user_predictions[user_col] = user_pred
        return user_predictions

In [16]:
# Centralized Traffic Forecaster
class CentralizedTrafficForecaster:
    def __init__(self, df, user_cols=None, timestamp_col='timestamp'):
        """
        Initialize centralized traffic forecaster.

        Args:
            df: DataFrame with traffic data for all users
            user_cols: List of column names for user traffic data (if None, auto-detect)
            timestamp_col: Name of the timestamp column
        """
        self.df = df.copy()
        self.timestamp_col = timestamp_col

        # Auto-detect user columns if not provided
        if user_cols is None:
            # Assume all columns except timestamp are user data
            self.user_cols = [col for col in df.columns if col != timestamp_col]
        else:
            self.user_cols = user_cols

        self.num_users = len(self.user_cols)
        print(f"Detected {self.num_users} users: {self.user_cols}")

        # Initialize data processor
        self.data_processor = CentralizedDataProcessor(
            df, self.user_cols, timestamp_col
        )

        # Initialize model
        self.model = CentralizedTrafficForecastingModel(
            num_users=self.num_users,
            input_dim=1,
            d_model=128,
            num_heads=8,
            d_ff=256,
            num_layers=4,
            dropout=0.1,
            pred_horizon= 24
        ).to(device)

        # Initialize optimizer and loss function
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.00001)
        self.criterion = nn.MSELoss()

    def train(self, start_date, end_date, batch_size=32, epochs=50):
        """
        Train the centralized model.

        Args:
            start_date: Start of training period
            end_date: End of training period
            batch_size: Batch size for training
            epochs: Number of epochs to train

        Returns:
            Training loss history
        """
        print(f"Training centralized model for {self.num_users} users...")

        # Prepare training data
        try:
            X_train, y_train = self.data_processor.prepare_training_data(
                start_date, end_date, seq_length=120, pred_horizon= 24
            )
        except Exception as e:
            print(f"Error preparing training data: {e}")
            return []

        # Create dataset and dataloader
        train_dataset = MultiUserTrafficDataset(X_train, y_train) # convert data format to tensor
        print("train_dataset",train_dataset)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        print("train_loader",train_loader)

        # Training history
        train_losses = []

        # Training loop
        self.model.train()
        for epoch in range(epochs):
            epoch_loss = 0.0
            for X_batch, y_batch in train_loader:
                # Move data to device
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)

                # Zero the gradients
                self.optimizer.zero_grad()

                # Forward pass
                outputs = self.model(X_batch)

                # Calculate loss
                loss = self.criterion(outputs, y_batch)

                # Backward pass and optimize
                loss.backward()
                self.optimizer.step()

                epoch_loss += loss.item()

            # Calculate average loss for the epoch
            avg_loss = epoch_loss / len(train_loader) if len(train_loader) > 0 else float('inf')
            train_losses.append(avg_loss)

            # Print progress
            print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}')

        return train_losses

    def predict(self, target_date):
        """
        Predict traffic for all users on a specific date.

        Args:
            target_date: The date to predict

        Returns:
            predictions: Dictionary of traffic predictions for each user
            target_hours: Hours corresponding to predictions
        """
        # Prepare input data
        model_input, target_hours = self.data_processor.prepare_input_for_date(target_date)

        # Convert to torch tensor
        model_input = torch.tensor(model_input, dtype=torch.float32).to(device)

        # Set model to evaluation mode
        self.model.eval()

        # Make prediction
        with torch.no_grad():
            predictions_scaled = self.model(model_input).cpu().numpy()
            print("predictions_scaled",predictions_scaled)

        # Convert back to original scale and split by user
        user_predictions = self.data_processor.inverse_transform_predictions(predictions_scaled[0])
        return user_predictions, target_hours # data users in each hour

    def plot_predictions(self, target_date):
        """
        Plot predictions for all users on a specific date.

        Args:
            target_date: The date to predict

        Returns:
            fig: Matplotlib figure
        """
        # Get predictions
        user_predictions, target_hours = self.predict(target_date)

        

        # Get actual data
        actual_data = {}
        try:
            actual_df = self.df[(self.df[self.timestamp_col] >= target_date) &
                              (self.df[self.timestamp_col] < target_date + timedelta(days=1))]

            for user_col in self.user_cols:
                if user_col in actual_df.columns and len(actual_df) == 24:
                    actual_data[user_col] = actual_df[user_col].values
        
        except Exception as e:
            print(f"Error getting actual data: {e}")
        
        
        # Create plot with subplots for each user
        fig, axes = plt.subplots(len(self.user_cols), 1, figsize=(15, 5 * len(self.user_cols)))

        # Handle case with only one user
        if len(self.user_cols) == 1:
            axes = [axes]

        # Plot each user's predictions
        for i, user_col in enumerate(self.user_cols):
            ax = axes[i]

            # Plot predictions
            ax.plot(target_hours, user_predictions[user_col], 'b-', label='Prediction')

            # Plot actual data if available
            if user_col in actual_data:
                ax.plot(target_hours, actual_data[user_col], 'r-', label='Actual')

            # Format subplot
            ax.set_title(f'Traffic Forecast for {user_col} on {pd.to_datetime(target_date).strftime("%Y-%m-%d")}')
            if i == len(self.user_cols) - 1:
                ax.set_xlabel('Hour')
            ax.set_ylabel('Traffic (Mbps)')
            ax.grid(True)
            ax.legend()

        plt.tight_layout()
        return fig

    def evaluate(self, start_date, end_date):
        """
        Evaluate model performance over a date range.

        Args:
            start_date: Start of evaluation period
            end_date: End of evaluation period

        Returns:
            Dictionary of evaluation metrics for each user
        """
        # Initialize metrics
        user_metrics = {user_col: {'rmse': [], 'mae': [], 'mape': []} for user_col in self.user_cols}
        dates = []

        # Evaluate each day in the range
        current_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)

        while current_date <= end_date:
            try:
                # Get predictions for all users
                user_predictions, _ = self.predict(current_date)

                # Get actual data
                actual_df = self.df[(self.df[self.timestamp_col] >= current_date) &
                                   (self.df[self.timestamp_col] < current_date + timedelta(days=1))]

                # Only evaluate if we have complete actual data
                if len(actual_df) == 148:
                    dates.append(current_date)

                    # Calculate metrics for each user
                    for user_col in self.user_cols:
                        if user_col in actual_df.columns:
                            actual_data = actual_df[user_col].values
                            predictions = user_predictions[user_col]

                            # Calculate metrics
                            rmse = np.sqrt(np.mean((predictions - actual_data) ** 2)) # ex: error mean([0.1, 0.2, 0.4]) 
                            mae = np.mean(np.abs(predictions - actual_data))
                            # Add small epsilon to avoid division by zero
                            mape = np.mean(np.abs((actual_data - predictions) / (actual_data + 1e-5))) * 100

                            # Store metrics
                            user_metrics[user_col]['rmse'].append(rmse)
                            user_metrics[user_col]['mae'].append(mae)
                            user_metrics[user_col]['mape'].append(mape)

                            print(f"Evaluated {user_col} on {current_date.date()}: RMSE={rmse:.2f}, MAE={mae:.2f}, MAPE={mape:.2f}%")

            except Exception as e:
                print(f"Error evaluating on {current_date.date()}: {e}")

            # Move to next day
            print("current_date",current_date)
            current_date += timedelta(days=1)

        # Calculate average metrics for each user
        avg_metrics = {}
        for user_col, metrics in user_metrics.items():
            avg_metrics[user_col] = {
                'avg_rmse': np.mean(metrics['rmse']) if metrics['rmse'] else None,
                'avg_mae': np.mean(metrics['mae']) if metrics['mae'] else None,
                'avg_mape': np.mean(metrics['mape']) if metrics['mape'] else None
            }

        return {
            'daily_metrics': dict(zip([d.date() for d in dates], user_metrics)),
            'avg_metrics': avg_metrics
        }

    def plot_evaluation_metrics(self, evaluation_results):
        """
        Plot average evaluation metrics for all users.

        Args:
            evaluation_results: Results from evaluate()

        Returns:
            fig: Matplotlib figure
        """
        if 'avg_metrics' not in evaluation_results:
            print("No average metrics available in evaluation results")
            return None

        # Extract metrics
        avg_metrics = evaluation_results['avg_metrics']

        # Create figure with subplots for different metrics
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        metrics = ['avg_rmse', 'avg_mae', 'avg_mape']
        titles = ['Average RMSE', 'Average MAE', 'Average MAPE (%)']

        # For each metric type
        for i, (metric, title) in enumerate(zip(metrics, titles)):
            ax = axes[i]

            # Extract values for this metric
            user_names = []
            metric_values = []

            for user_col, user_avg_metrics in avg_metrics.items():
                if user_avg_metrics[metric] is not None:
                    user_names.append(user_col)
                    metric_values.append(user_avg_metrics[metric])

            # Create bar chart
            if metric_values:
                ax.bar(user_names, metric_values)
                ax.set_title(title)
                ax.set_ylabel(title.split()[1])
                ax.set_xlabel('User')
                ax.grid(True, axis='y')

                # Rotate x-labels if there are many users
                if len(user_names) > 5:
                    ax.set_xticklabels(user_names, rotation=45, ha='right')

        plt.tight_layout()
        return fig

    def save_model(self, filepath):
        """Save the model to disk."""
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict()
        }, filepath)
        print(f"Model saved to {filepath}")

    def load_model(self, filepath):
        """Load the model from disk."""
        checkpoint = torch.load(filepath)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(f"Model loaded from {filepath}")


In [ ]:
# Example usage with synthetic data
if __name__ == "__main__":
    print("Generating synthetic traffic data for multiple users...")

    # Generate data for 5 users over 3 months
    num_users = 5
    start_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=60)
    df, user_patterns = generate_multi_user_data(
        num_users=num_users,
        base_lambda=15,  # 10 Base traffic rate
        num_hours=3*720,  # 3 months of hourly data
        start_date=start_date,
        seed=42  # For reproducibility
    )

    # Save generated data to CSV
    df.to_csv('multi_user_traffic_data.csv', index=False)
    print(f"Saved synthetic data to multi_user_traffic_data.csv")

    # Plot sample of the generated data for each user
    plt.figure(figsize=(15, 8))
    for i in range(1, num_users + 1):
        user_col = f"user_{i}"
        plt.plot(df['timestamp'][:720], df[user_col][:720], label=user_col)  # Plot first month

    plt.title('Generated Traffic Data for Multiple Users (First Month)')
    plt.xlabel('Time')
    plt.ylabel('Traffic (Mbps)')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('multi_user_traffic_sample.png')
    plt.show()

    # Initialize centralized forecaster
    forecaster = CentralizedTrafficForecaster(df)

    # Define training and evaluation periods
    train_start = start_date  # previouse two month
    train_end = start_date + timedelta(days=30)  # First month
    eval_start = start_date + timedelta(days=35)  # A few days into second month
    eval_end = start_date + timedelta(days=45)  # 10-day evaluation period

    # Train the centralized model
    print(f"\nTraining centralized model on data from {train_start.date()} to {train_end.date()}")
    training_losses = forecaster.train(
        train_start,
        train_end,
        batch_size=64,
        epochs=500  # Use more epochs (e.g., 50) for better results
    )

    # Plot training loss
    plt.figure(figsize=(10, 6))
    plt.plot(training_losses)
    plt.title('Centralized Model Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('MSE Loss')
    plt.grid(True)
    plt.savefig('centralized_model_training_loss.png')
    plt.show()

    # Make predictions for a specific date
    target_date = eval_start
    print(f"\nGenerating predictions for all users on {target_date.date()}")

    # Plot predictions
    fig = forecaster.plot_predictions(target_date)
    plt.savefig('centralized_model_predictions.png')
    plt.show()

    # Evaluate model on test period
    print(f"\nEvaluating centralized model from {eval_start.date()} to {eval_end.date()}")
    evaluation_results = forecaster.evaluate(eval_start, eval_end)

    # Print summary of evaluation results
    if 'avg_metrics' in evaluation_results:
        print("\nEvaluation Summary:")
        for user_col, metrics in evaluation_results['avg_metrics'].items():
            print(f"{user_col}: RMSE={metrics['avg_rmse']:.2f}, MAE={metrics['avg_mae']:.2f}, MAPE={metrics['avg_mape']:.2f}%")

    # Plot evaluation metrics
    fig = forecaster.plot_evaluation_metrics(evaluation_results)
    plt.savefig('centralized_model_evaluation.png')
    plt.show()

    # Save model
    forecaster.save_model('centralized_traffic_forecaster.pt')

    print("\nCentralized traffic forecasting completed!")

In [ ]:
# forecaster.load_model(('centralized_traffic_forecaster.pt'))
# model.eval()
target_date = datetime(2025, 2, 17, 0, 0)
user_predictions, target_hours = forecaster.predict(target_date)
type(user_predictions)

In [ ]:
df = pd.DataFrame(user_predictions, index=target_hours)
print(df)
df.to_excel("user_predictions.xlsx", index=False)

In [ ]:
### user_row, 24hour_column
user_dic = user_predictions
user_array = list(user_dic.values())
all_user = np.stack(user_array)
all_user[:,16] # all user in time 16